In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Concatenate, Reshape
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
open_channels = df_train.open_channels
df_train = pd.get_dummies(df_train, prefix = 'oc',columns=['open_channels'])
df_train['open_channels'] = open_channels
df_train

,time,signal,binary_open_channels,oc_0,oc_1,oc_2,oc_3,oc_4,oc_5,oc_6,oc_7,oc_8,oc_9,oc_10,open_channels
0,0.0001,-2.7600,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.0002,-2.8557,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.0003,-2.4074,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0.0004,-3.1404,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0.0005,-3.1525,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,499.9996,2.9194,1,0,0,0,0,0,0,0,1,0,0,0,7
4999996,499.9997,2.6980,1,0,0,0,0,0,0,0,1,0,0,0,7
4999997,499.9998,4.5164,1,0,0,0,0,0,0,0,0,1,0,0,8
4999998,499.9999,5.6397,1,0,0,0,0,0,0,0,0,0,1,0,9


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train)
categorical_output = df_train[['oc_0','oc_1','oc_2','oc_3','oc_4','oc_5','oc_6','oc_7','oc_8','oc_9','oc_10']]

data = np.zeros((len(signal), 12))
data[:, 0] = signal
data[:, 1:] = categorical_output
data = data.reshape((1000, 5000, 12))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1:].reshape((1000,5000,11))

In [4]:
np.shape(x),np.shape(y)

((1000, 5000, 1), (1000, 5000, 11))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [12]:
inputs = Input(shape=(5000,1,))

layer1_a = LSTM(units=3, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_a = Dense(5,activation='relu')(layer1_a)

layer1_b = LSTM(units=5, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_b = Dense(8,activation='relu')(layer1_b)

layer1_c = SimpleRNN(units=10, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_c = Dense(16,activation='relu')(layer1_c)
concat = Concatenate()([layer2_a, layer2_b,layer2_c])

outputs = Dense(units=11, activation='softmax')(concat)

In [13]:
model = Model(inputs,outputs)
opt = Adam(learning_rate=.01,beta_1=0.95)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 5000, 1)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 5000, 3)      60          input_3[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 5000, 5)      140         input_3[0][0]                    
__________________________________________________________________________________________________
simple_rnn_6 (SimpleRNN)        (None, 5000, 10)     120         input_3[0][0]                    
____________________________________________________________________________________________

In [15]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 67s 96ms/sample - loss: 1.3622 - accuracy: 0.4945 - val_loss: 1.3260 - val_accuracy: 0.4970
Epoch 2/10
700/700 [==============================] - 73s 105ms/sample - loss: 1.3402 - accuracy: 0.4815 - val_loss: 1.2978 - val_accuracy: 0.4969
Epoch 3/10
700/700 [==============================] - 104s 149ms/sample - loss: 1.3150 - accuracy: 0.5006 - val_loss: 1.2805 - val_accuracy: 0.5163
Epoch 4/10
700/700 [==============================] - 94s 135ms/sample - loss: 1.2860 - accuracy: 0.5206 - val_loss: 1.2559 - val_accuracy: 0.5312
Epoch 5/10
700/700 [==============================] - 91s 130ms/sample - loss: 1.2614 - accuracy: 0.5292 - val_loss: 1.2149 - val_accuracy: 0.5661
Epoch 6/10
700/700 [==============================] - 87s 124ms/sample - loss: 1.2280 - accuracy: 0.5404 - val_loss: 1.2043 - val_accuracy: 0.5523
Epoch 7/10
700/700 [==============================] - 86s 122ms/sample -

In [16]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/5
700/700 [==============================] - 89s 127ms/sample - loss: 1.0982 - accuracy: 0.6008 - val_loss: 1.0837 - val_accuracy: 0.6076
Epoch 2/5
700/700 [==============================] - 101s 145ms/sample - loss: 1.0829 - accuracy: 0.6100 - val_loss: 1.0376 - val_accuracy: 0.6351
Epoch 3/5
700/700 [==============================] - 86s 123ms/sample - loss: 1.0448 - accuracy: 0.6224 - val_loss: 1.0148 - val_accuracy: 0.6431
Epoch 4/5
700/700 [==============================] - 89s 127ms/sample - loss: 1.0247 - accuracy: 0.6302 - val_loss: 1.0041 - val_accuracy: 0.6437
Epoch 5/5
700/700 [==============================] - 87s 124ms/sample - loss: 1.0955 - accuracy: 0.6172 - val_loss: 1.0091 - val_accuracy: 0.6497


In [19]:
results = model.predict(x)

In [27]:
correct = 0
incorrect = 0
for i in range(len(results)):
    for j in range(len(results[0])):
        a = np.where(results[i][j] == np.amax(results[i][j]))[0]
        b = np.where(y[i][j] == np.amax(y[i][j]))[0]
        if a[0] == b[0]:
            correct += 1
        else:
            incorrect += 1
print(correct/(correct+incorrect))

0.6363824


In [29]:
model.save('../Models/multi_path_model_63.h5')